# Does a local change cause a change in local crime rates?

The eternal complaint of people in my neighborhood regarding just about any change that may help the homeless seems to be that it will increase crime (at least to judge from NextDoor). So I had the question: does it?

The standard complaint for just about any change in legislative or policing policy seems to be something along the lines of "Ever since {change} was implemented, {crime in general/specific crime subcategory} has increased in my {block/neighborhood/area}."

So, can I create a relatively simple tool that will take some basic user input (when and where) and run a comparison to figure out whether crime has really increased?

I'm going to be using pandas to manipulate the data and seaborn to display the results.

In [3]:
import pandas as pd
import seaborn as sb
#event_location =
#event_date =

## The police dataset - onboarding and cleaning up
The SPD publishes a .csv of virtually all police reports, with the following columns:
* Report Number
* Occurred Date
* Occurred Time
* Reported Date
* Reported Time
* Crime Subcategory
* Primary Offense Description
* Precinct
* Sector
* Beat
* Neighborhood

For development, I'm using a download of the police reports. (For actual implementation, it is possible to download new data on demand as the tool is run, determining if the most recent report date is within the date range used and downloading all reports reported after that date and all non-redundant reports from that report date.)

I'm dropping times from the dataframe as they're unneeded and converting the date strings to datetime. This conversion takes a non-trivial amount of time, hence the future plan to download only new data, convert, and add to the dataframe.

In [21]:
crime_data = pd.read_csv('Crime_Data.csv')
#Dropping times from database
crime_data.drop(columns = ["Occurred Time", "Reported Time"], inplace = True)
#Converting dates to datetime objects
crime_data[["Occurred Date", "Reported Date"]] = crime_data[["Occurred Date", "Reported Date"]].apply(pd.to_datetime)
crime_data

,Report Number,Occurred Date,Reported Date,Crime Subcategory,Primary Offense Description,Precinct,Sector,Beat,Neighborhood
0,1975000079415,1975-12-16,1975-12-16,BURGLARY-RESIDENTIAL,BURGLARY-FORCE-RES,SOUTH,R,R3,LAKEWOOD/SEWARD PARK
1,1976000069169,1976-01-01,1976-01-31,SEX OFFENSE-OTHER,SEXOFF-INDECENT LIBERTIES,UNKNOWN,NaN,NaN,UNKNOWN
2,1979000049118,1979-01-28,1979-02-09,CAR PROWL,THEFT-CARPROWL,EAST,G,G2,CENTRAL AREA/SQUIRE PARK
3,19810000305573,1981-08-22,1981-08-22,HOMICIDE,HOMICIDE-PREMEDITATED-WEAPON,SOUTH,S,S2,BRIGHTON/DUNLAP
4,1981000076456,1981-02-14,1981-02-15,BURGLARY-RESIDENTIAL,BURGLARY-FORCE-RES,SOUTHWEST,W,W3,ROXHILL/WESTWOOD/ARBOR HEIGHTS
5,19880000507482,1988-09-29,1988-09-29,MOTOR VEHICLE THEFT,VEH-THEFT-AUTO,WEST,M,M2,SLU/CASCADE
6,19930000465323,1993-10-08,1993-10-08,HOMICIDE,HOMICIDE-PREMEDITATED-GUN,SOUTH,R,R2,CLAREMONT/RAINIER VISTA
7,19940000264046,1994-06-08,1994-06-12,THEFT-ALL OTHER,THEFT-OTH,SOUTHWEST,F,F1,HIGH POINT
8,19960000543964,1996-12-08,1996-12-08,CAR PROWL,THEFT-CARPROWL,SOUTH,O,O1,SODO
9,19990000333737,NaT,1999-01-01,THEFT-SHOPLIFT,THEFT-SHOPLIFT,UNKNOWN,NaN,NaN,UNKNOWN


Where occurred date is unknown, there are two approaches possible: drop those entries OR fill those values with another reasonable value.

The simplest solution seems to be filling in cases where occurred date = NaT with the reported date. (There are no entries without a reported date)

In [23]:
crime_data["Occurred Date"] = crime_data["Occurred Date"].fillna(value = crime_data["Reported Date"])
crime_data

,Report Number,Occurred Date,Reported Date,Crime Subcategory,Primary Offense Description,Precinct,Sector,Beat,Neighborhood
0,1975000079415,1975-12-16,1975-12-16,BURGLARY-RESIDENTIAL,BURGLARY-FORCE-RES,SOUTH,R,R3,LAKEWOOD/SEWARD PARK
1,1976000069169,1976-01-01,1976-01-31,SEX OFFENSE-OTHER,SEXOFF-INDECENT LIBERTIES,UNKNOWN,NaN,NaN,UNKNOWN
2,1979000049118,1979-01-28,1979-02-09,CAR PROWL,THEFT-CARPROWL,EAST,G,G2,CENTRAL AREA/SQUIRE PARK
3,19810000305573,1981-08-22,1981-08-22,HOMICIDE,HOMICIDE-PREMEDITATED-WEAPON,SOUTH,S,S2,BRIGHTON/DUNLAP
4,1981000076456,1981-02-14,1981-02-15,BURGLARY-RESIDENTIAL,BURGLARY-FORCE-RES,SOUTHWEST,W,W3,ROXHILL/WESTWOOD/ARBOR HEIGHTS
5,19880000507482,1988-09-29,1988-09-29,MOTOR VEHICLE THEFT,VEH-THEFT-AUTO,WEST,M,M2,SLU/CASCADE
6,19930000465323,1993-10-08,1993-10-08,HOMICIDE,HOMICIDE-PREMEDITATED-GUN,SOUTH,R,R2,CLAREMONT/RAINIER VISTA
7,19940000264046,1994-06-08,1994-06-12,THEFT-ALL OTHER,THEFT-OTH,SOUTHWEST,F,F1,HIGH POINT
8,19960000543964,1996-12-08,1996-12-08,CAR PROWL,THEFT-CARPROWL,SOUTH,O,O1,SODO
9,19990000333737,1999-01-01,1999-01-01,THEFT-SHOPLIFT,THEFT-SHOPLIFT,UNKNOWN,NaN,NaN,UNKNOWN


## Changes in beats
One of the complications of using this dataset is that the most precise location is by beat, and beat boundaries have changed repeatedly over the years.

Crime data is noisy, particularly violent crime, since the numbers are relatively low. At a guess, a 90 day window is probably the minimum span where the signal-to-noise ratio is acceptable. Given that there are seasonal variations in all kinds of human data (and things like noise complaints are probably more prevalent in the summer), the default will be to compare the year prior to the given date and the year following the given date.

There have also been small changes to how precincts and sectors have been drawn, but precinct-level changes seem to be limited to the East and West Precincts for 2015.

Per the SPD Beats data, https://data.seattle.gov/Public-Safety/Seattle-Police-Department-Beats/nnxn-434b, beats have been changed at 2008, 2015, and 2018. For the moment, comparisons that cross one of these date boundaries will throw a warning that data may be flawed as beat boundaries may have changed; with more time to research the history, analyses can be allowed or disallowed based on whether or not the boundaries of the beat in question changed at the particular time.

In [35]:
import datetime
window = 90
event_date = datetime.datetime(2009, 3, 5)
beat_changes = [datetime.datetime(2008, 1, 1), datetime.datetime(2015, 1, 1), datetime.datetime(2018, 1, 1)]
for entry in beat_changes:
    if abs(event_date - entry) < datetime.timedelta(days = window):
        print("CAUTION: Beats have changed during the window used; proceed with caution or compare by sector/precinct")

## Binning crime into types
Since crime complaints are usually things like "there's more crime in the neighborhood!" or "there's more car prowls!", I can increase the power of the dataset by binning crime types together. There are 30 unique crime subtypes in the dataset, with 259 entries having the crime subcategory left blank.

These can be combined into:
* Violent (assault, homicide, rape, robbery, sex offense-other, weapon)
* Property (arson, burglary, motor vehicle theft, theft)
* Law and order (car prowl, disorderly conduct, family offense-nonviolent, gamble, liquor law violation, loitering, narcotic, pornography, prostitution, trespass)
* Location-unrelated (DUI)

Note: robbery is defined as taking something directly from someone through violence or threat of violence, hence its binning under "Violent"

At the time this was originally developed, there were approximately 250k property crimes, 200k L&O crimes, 50k violent crimes, and 12k DUI, with 259 blank.

In [50]:
crime_bin = {"AGGRAVATED ASSAULT": "Violent", "AGGRAVATED ASSAULT-DV": "Violent", "HOMICIDE": "Violent", "RAPE": "Violent", "ROBBERY-COMMERCIAL": "Violent", "ROBBERY-RESIDENTIAL": "Violent", "ROBBERY-STREET": "Violent", "SEX OFFENSE-OTHER": "Violent", "WEAPON": "Violent",
    "ARSON": "Property", "BURGLARY-COMMERCIAL": "Property", "BURGLARY-COMMERCIAL-SECURE PARKING": "Property", "BURGLARY-RESIDENTIAL": "Property", "BURGLARY-RESIDENTIAL-SECURE PARKING": "Property", "MOTOR VEHICLE THEFT": "Property", "THEFT-ALL OTHER": "Property", "THEFT-BICYCLE": "Property", "THEFT-BUILDING": "Property", "THEFT-SHOPLIFT": "Property",
    "CAR PROWL": "Law and Order", "DISORDERLY CONDUCT": "Law and Order", "FAMILY OFFENSE-NONVIOLENT": "Law and Order", "GAMBLE": "Law and Order", "LIQUOR LAW VIOLATION": "Law and Order", "LOITERING": "Law and Order", "NARCOTIC": "Law and Order", "PORNOGRAPHY": "Law and Order", "PROSTITUTION": "Law and Order", "TRESPASS": "Law and Order",
    "DUI": "Unrelated",
    None: "Undefined"}


From the parent crime_data set, a subset is constructed consisting of incidents within the query window of the date in question with types binned into the four types above. Those entries where the crime subcategory was left blank are binned as "Undefined" - these are all described as "burglary-other" or "robbery-other", meaning they can't be binned in the same category and are likely enough of an edge case to be ignorable

In [52]:
start = event_date - datetime.timedelta(days = window)
end = event_date + datetime.timedelta(days = window)
within_window = crime_data.loc[(crime_data["Occurred Date"] > start) & (crime_data["Occurred Date"] < end), ["Report Number", "Occurred Date", "Crime Subcategory", "Precinct", "Sector", "Beat"]].copy()
within_window["Crime Subcategory"] = within_window["Crime Subcategory"].map(crime_bin)
def before_after(occurreddate):
    if occurreddate < event_date:
        return "Before"
    else:
        return "After"
within_window["Occurred Date"] = within_window["Occurred Date"].map(before_after)
within_window

,Report Number,Occurred Date,Crime Subcategory,Precinct,Sector,Beat
16207,20080000268060,Before,Unrelated,NORTH,J,J1
31641,20080000421341,Before,Law and Order,NORTH,B,B1
34710,20080000450172,Before,Property,NORTH,L,L3
35099,20080000454248,Before,Law and Order,WEST,K,K1
35166,20080000455080,Before,Property,NORTH,N,N3
35173,20080000455172,Before,Law and Order,NORTH,B,B2
35174,20080000455176,Before,Law and Order,WEST,K,K2
35176,20080000455188,Before,Law and Order,NORTH,B,B3
35177,20080000455210,Before,Violent,WEST,M,M3
35181,20080000455276,Before,Unrelated,NORTH,B,B3


## Getting rates

From the subset, we can get a count of crimes of each type, before and after the date in question, within and without the area in question.

The simplest comparison is to look at the percentage change in crime from the before to the after, comparing the within-beat change to the change for all other beats across the same time period.

Comparing crime from area to area is often done on #/standard population unit level, but the census data I found isn't easily applied to beats, and census data is going to be a bit squiffy when it comes to actual occupancy in areas not zoned for residences (i.e. lots of people work there during the day and a few people live there in their cars at night).

In [62]:
precinct = ["EAST", "NORTH", "SOUTH", "SOUTHWEST", "WEST", "UNKNOWN"]
sector = ["B", "C", "D", "E", "F", "G", "J", "K", "L", "M", "N", "O", "Q", "R", "S", "U", "W"]
beat = ["B1", "B2", "B3", "C1", "C2", "C3", "D1", "D2", "D3", "E1", "E2", "E3", "F1", "F2", "F3", "G1", "G2", "G3", "J1", "J2", "J3", "K1", "K2", "K3", "L1", "L2", "L3", "M1", "M2", "M3", "N1", "N2", "N3", "O1", "O2", "O3", "Q1", "Q2", "Q3", "R1", "R2", "R3", "S1", "S2", "S3", "U1", "U2", "U3", "W1", "W2", "W3"]

#Using sum to get count of entries that fit given parameters by location (geo_flag)
#geo_flag manually set to beat here, but future plan is to allow user to choose at onset, with beat being default
geo_flag = beat
all_before = [sum((within_window["Occurred Date"] == "Before") & (within_window["Beat"] == entry)) for entry in geo_flag]
all_after = [sum((within_window["Occurred Date"] == "After") & (within_window["Beat"] == entry)) for entry in geo_flag]
all_percent = [(after-before)/(max(before, 1))*100 for before, after in zip(all_before, all_after)]
all_percent

[-6.914893617021277,
 -19.93127147766323,
 -21.824104234527688,
 16.93548387096774,
 11.818181818181818,
 5.454545454545454,
 3.6585365853658534,
 14.093959731543624,
 16.964285714285715,
 -10.650887573964498,
 -5.084745762711865,
 16.582914572864322,
 11.450381679389313,
 21.804511278195488,
 16.89189189189189,
 4.807692307692308,
 -6.382978723404255,
 18.27956989247312,
 -19.3717277486911,
 -39.166666666666664,
 -13.5,
 32.82051282051282,
 11.162790697674419,
 0.35714285714285715,
 -14.285714285714285,
 -17.78523489932886,
 -2.515723270440252,
 64.24870466321244,
 3.149606299212598,
 21.338912133891213,
 0.7407407407407408,
 -8.19672131147541,
 -12.650602409638553,
 45.78313253012048,
 -20.77922077922078,
 -8.13953488372093,
 31.147540983606557,
 -28.74015748031496,
 24.350649350649352,
 15.348837209302326,
 -4.9504950495049505,
 18.867924528301888,
 -20.28301886792453,
 -4.700854700854701,
 2.5974025974025974,
 -2.6119402985074625,
 10.619469026548673,
 40.28436018957346,
 6.8421052

However, there is going to be noise in the system. It may be helpful to look at if the percentage change in reports in that area is significantly different from the same data for all areas in that time period.

Of course, if the percentage change in crime is not statistically different for that area compared to other areas, hypotheses will tend to arise that propose the rise in crime is seen in specific types of crime. The data binning occurred at the creation of the subset to discourage users from flailing for changes that hit statistical significance by testing an excessive number of simultaneous hypotheses, though somebody with any proficiency in reading Python could readily delete that portion of the code.

That said, comparing the change in, for example, narcotics within the beat to the change in narcotics for all beats will thwart the worst of these instincts. "Noisy" crime types (those with relatively few reports where a single report can have an outsize effect on the percentage displayed) will be noisy across most if not all beats. The standard deviation of percentage change will therefore be large, and the likelihood of one beat falling outside three sigma by chance will remain small.